In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
import warnings
warnings.simplefilter("ignore")

In [3]:
df = pd.read_csv("covid_toy.csv")

In [4]:
df.sample(5)

,age,gender,fever,cough,city,has_covid
15,70,Male,103.0,Strong,Kolkata,Yes
36,38,Female,101.0,Mild,Bangalore,No
81,65,Male,99.0,Mild,Delhi,No
94,79,Male,NaN,Strong,Kolkata,Yes
99,10,Female,98.0,Strong,Kolkata,Yes


In [5]:
df["gender"].value_counts()

gender
Female    59
Male      41
Name: count, dtype: int64

In [6]:
df["cough"].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [7]:
df["has_covid"].value_counts()

has_covid
No     55
Yes    45
Name: count, dtype: int64

In [8]:
 df["city"].value_counts()

city
Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: count, dtype: int64

In [9]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [10]:
from sklearn.model_selection import train_test_split

In [40]:
x= df.drop(["has_covid"], axis=1)
y= df["has_covid"]

In [41]:
y, x

(0      No
 1     Yes
 2      No
 3      No
 4      No
      ... 
 95     No
 96    Yes
 97     No
 98     No
 99    Yes
 Name: has_covid, Length: 100, dtype: object,
     age  gender  fever   cough       city
 0    60    Male  103.0    Mild    Kolkata
 1    27    Male  100.0    Mild      Delhi
 2    42    Male  101.0    Mild      Delhi
 3    31  Female   98.0    Mild    Kolkata
 4    65  Female  101.0    Mild     Mumbai
 ..  ...     ...    ...     ...        ...
 95   12  Female  104.0    Mild  Bangalore
 96   51  Female  101.0  Strong    Kolkata
 97   20  Female  101.0    Mild  Bangalore
 98    5  Female   98.0  Strong     Mumbai
 99   10  Female   98.0  Strong    Kolkata
 
 [100 rows x 5 columns])

In [42]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2, random_state = 0)

1 AAM ZINDAGI

# adding sample imputer to fever col

In [14]:
si = SimpleImputer()

In [15]:
x_train_fever = si.fit_transform(x_train[["fever"]])

In [16]:
# also the test data

In [17]:
x_test_fever = si.fit_transform(x_test[["fever"]])

In [18]:
x_train_fever.shape

(80, 1)

# ordinal encoding - cough

In [19]:
oe = OrdinalEncoder (categories = [["Mild", "Strong"]])

In [20]:
x_train_cough = oe.fit_transform(x_train[["cough"]])
x_test_cough = oe.fit_transform(x_test[["cough"]])

In [21]:
x_test_cough

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.]])

# ONE HOT ENCODING 

In [22]:
ohe = OneHotEncoder(drop = "first", sparse_output = False)

In [23]:
x_train_gender_city = ohe.fit_transform(x_train[["gender",'city']])
x_test_gender_city = ohe.fit_transform(x_test[["gender", "city"]])

# EXTRACTING AGE

In [24]:
x_train_age = x_train.drop(columns = ["gender","fever","cough","city"]).values
x_test_age = x_test.drop(columns = ["gender","fever","cough","city"]).values
x_train_age.shape

(80, 1)

In [25]:
x_train_trans = np.concatenate((x_train_age,x_train_gender_city,x_train_cough,x_train_fever),axis=1)
x_test_trans = np.concatenate((x_test_age,x_test_gender_city,x_test_cough,x_test_fever),axis=1)

In [26]:
x_train_trans.shape

(80, 7)

# MENTOS ZINDAGI

In [27]:
from sklearn.compose import ColumnTransformer

In [29]:
transformer = ColumnTransformer(transformers = [
    ("tnf1",SimpleImputer(),["fever"]),
    ("tnf2",OrdinalEncoder(categories = [["Mild","Strong"]]),["cough"]),
    ("tnf3",OneHotEncoder(sparse_output= False, drop = "first"),["gender",'city'])
],remainder = "passthrough")

In [45]:
x_train = transformer.fit_transform(x_train)

In [49]:
transformer.transform(x_train).shape

ValueError: X has 7 features, but ColumnTransformer is expecting 5 features as input.